# Opis notatnika

Jest to pierwszy notatnik, na któym przeprowadzane były testy podglądowe. Występują tutaj stare wersje niektórych funkcji, które służyły wypisywaniu statystyk. Zostało to zamienione na generowanie raportów pdf z danych eksperymentów, ale żbey nie tracić wyników uzyskanych w dotychczasowych próbach, zostawiłem ten notatnik.

In [1]:
import sys
from pathlib import Path
import os


SRC = (Path("..")).resolve()
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from classes.pokemon_team import PokemonTeam
from simulation.simulation import simulate_battle
from simulation.formulas import multiply_type_multiplier, damage_attack_devide_defense
from data.data import get_pokemons
from solvers import SimulatedAnnealingPokemonSolver
from solvers import HillClimbingPokemonSolver
from solvers import RandomSearchPokemonSolver
from pandera.typing import DataFrame
import numpy as np
from pandera.typing import DataFrame
from classes.pokemon_team import PokemonTeam
from data import get_pokemons
from schemas import PokemonSchema


In [3]:
pokemons = get_pokemons()  # Ostatnie ewolucje

print("Liczba pokemonów:", len(pokemons))
print("Kolumny:", list(pokemons.columns)[:20], "...")
display(pokemons.head(3))

Liczba pokemonów: 359
Kolumny: ['against_bug', 'against_dark', 'against_dragon', 'against_electric', 'against_fairy', 'against_fighting', 'against_fire', 'against_flying', 'against_ghost', 'against_grass', 'against_ground', 'against_ice', 'against_normal', 'against_poison', 'against_psychic', 'against_rock', 'against_steel', 'against_water', 'attack', 'defense'] ...


,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fighting,against_fire,against_flying,against_ghost,against_grass,...,defense,hp,name,pokedex_number,sp_attack,sp_defense,speed,type1,type2,is_legendary
2,1.00,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,0.25,...,123,80,Venusaur,3,122,120,80,grass,poison,0
5,0.25,1.0,1.0,2.0,0.5,0.5,0.5,1.0,1.0,0.25,...,78,78,Charizard,6,159,115,100,fire,flying,0
8,1.00,1.0,1.0,2.0,1.0,1.0,0.5,1.0,1.0,2.00,...,120,79,Blastoise,9,135,115,78,water,NaN,0


In [13]:
t1_names = ['Snorlax', 'Salamence', 'Gigalith', 'Wobbuffet', 'Lapras', 'Galvantula']
t2_names = ['Gogoat', 'Rhyperior', 'Ampharos', 'Blissey', 'Drifblim', 'Wailord']

In [ ]:
for name in t1_names:
    members_t1 = pokemons[pokemons['name'].isin(t1_names)]
team1 = PokemonTeam(members_t1)

for name in t2_names:
    members_t2 = pokemons[pokemons['name'].isin(t2_names)]
team2 = PokemonTeam(members_t2)

result = simulate_battle(team1, team2,     multiply_type_multiplier,
    damage_attack_devide_defense,)

print("Team1:", team1)
print("Team2:", team2)
print("Remaining HP team1:", result, " / ", sum(team1.get_hps()))
print("Remaining ratio:", result / sum(team1.get_hps()))
print("Team2 remaining HP:", sum(team2.get_hps()))

Team1: PokemonTeam(size=6, names=['Lapras', 'Snorlax', 'Wobbuffet', 'Salamence', 'Gigalith', 'Galvantula'])
Team2: PokemonTeam(size=6, names=['Ampharos', 'Blissey', 'Wailord', 'Drifblim', 'Rhyperior', 'Gogoat'])
Remaining HP team1: 0  /  730
Remaining ratio: 0.0
Team2 remaining HP: 903


In [ ]:
result = simulate_battle(team2, team1,     multiply_type_multiplier,
    damage_attack_devide_defense,)

print("Team1:", team1)
print("Team2:", team2)
print("Remaining HP team2:", result, " / ", sum(team2.get_hps()))
print("Remaining ratio:", result / sum(team2.get_hps()))

Team1: PokemonTeam(size=6, names=['Lapras', 'Snorlax', 'Wobbuffet', 'Salamence', 'Gigalith', 'Galvantula'])
Team2: PokemonTeam(size=6, names=['Ampharos', 'Blissey', 'Wailord', 'Drifblim', 'Rhyperior', 'Gogoat'])
Remaining HP team2: 403  /  903
Remaining ratio: 0.44629014396456257
Team2 remaining HP: 730


In [8]:
t1 = PokemonTeam.generate_team(pokemons, unique_types=True)
t2 = PokemonTeam.generate_team(pokemons, unique_types=True)

remaining = simulate_battle(
    t1,
    t2,
    multiply_type_multiplier,
    damage_attack_devide_defense,
)

print("Team1:", t1)
print("Team2:", t2)
print("Remaining HP team1:", remaining, " / ", sum(t1.get_hps()))
print("Remaining ratio:", remaining / sum(t1.get_hps()))

Team1: PokemonTeam(size=6, names=['Salamence', 'Whiscash', 'Carnivine', 'Comfey', 'Eelektross', 'Pyroar'])
Team2: PokemonTeam(size=6, names=['Wigglytuff', 'Electrode', 'Garchomp', 'Aurorus', 'Pelipper', 'Parasect'])
Remaining HP team1: 0  /  501
Remaining ratio: 0.0


### Porównanie 3 solverów

In [15]:
TEAM_SIZE = 6
OPP_LIMIT = 30
BUDGET = 50


opponents = PokemonTeam.generate_unique_teams(
    pokemons,
    opponents_limit=OPP_LIMIT,
    max_attempts=OPP_LIMIT * 3,
    team_size=TEAM_SIZE,
    unique_types=True,
)
print("Opponents:", len(opponents))

start_team = PokemonTeam.generate_team(pokemons, team_size=TEAM_SIZE, unique_types=True)
print("Start team:", start_team)
print("Start team stats sum:", start_team.get_stats_sum())

Opponents: 30
Start team: PokemonTeam(size=6, names=['Gengar', 'Crabominable', 'Ferrothorn', 'Chatot', 'Gorebyss', 'Turtonator'])
Start team stats sum: 2154


In [10]:
sa = SimulatedAnnealingPokemonSolver(
    initial_temperature=0.5,
    min_temperature=1e-3,
    alpha=0.995,
    iters_per_temp=30,
    max_evaluations=BUDGET,
    neighbor_replacements=1,
    opponents_limit=OPP_LIMIT,
    unique_types=True,
    seed=123,
    patience=10,
    restarts=0,  # ważne w kontrolowanym starcie
)


sa_t0 = SimulatedAnnealingPokemonSolver(
    initial_temperature=1e-12,
    min_temperature=1e-13,
    alpha=0.5,
    iters_per_temp=30,
    max_evaluations=BUDGET,
    neighbor_replacements=1,
    opponents_limit=OPP_LIMIT,
    unique_types=True,
    seed=123,
    patience=10,
    restarts=0,
)

# 3) Hill Climbing
hc = HillClimbingPokemonSolver(
    max_evaluations=BUDGET,
    neighbors_per_step=30,
    neighbor_replacements=1,
    opponents_limit=OPP_LIMIT,
    unique_types=True,
    seed=123,
    restarts=0,  # ważne w kontrolowanym starcie
    patience=10,
)


rs = RandomSearchPokemonSolver(
    trials=BUDGET, opponents_limit=OPP_LIMIT, unique_types=True
)

In [ ]:
import inspect

def run_solver_with_optional_start_and_opponents(
    solver, pokemons, opponents, start_team=None
):
    sig = inspect.signature(solver.solve)
    kwargs = {}
    if "opponents" in sig.parameters:
        kwargs["opponents"] = opponents
    if start_team is not None and "start_team" in sig.parameters:
        kwargs["start_team"] = start_team
    return solver.solve(pokemons, **kwargs)

In [12]:
# --- Run: SA i HC startują z tego samego teamu ---
sa_team, sa_fit, sa_hist, _ = run_solver_with_optional_start_and_opponents(
    sa, pokemons, opponents, start_team=start_team
)

hc_team, hc_fit, hc_hist, _ = run_solver_with_optional_start_and_opponents(
    hc, pokemons, opponents, start_team=start_team
)

sa0_team, sa0_fit, sa0_hist, _ = run_solver_with_optional_start_and_opponents(
    sa_t0, pokemons, opponents, start_team=start_team
)

rs_team, rs_fit, rs_hist, _ = rs.solve(pokemons, opponents)


print("\nRESULTS:")
print("SA fitness:", sa_fit)
print("HC fitness:", hc_fit)
print("SA(temp~0) fitness:", sa0_fit)
print("RS fitness:", rs_fit)


RESULTS:
SA fitness: 0.19443339960238573
HC fitness: 0.09570093457943925
SA(temp~0) fitness: 0.2195638629283489
RS fitness: 0.20683760683760685


In [ ]:
def run_multiple_runs(
    pokemons: DataFrame[PokemonSchema],
    solver: SimulatedAnnealingPokemonSolver,
    runs: int = 8,
    opponents: list[PokemonTeam] = None,
) -> None:
    print(f"Running {runs} runs of the SA solver...")

    best_teams: list[PokemonTeam] = []
    best_fitnesses: list[float] = []

    for i in range(runs):
        print(f"Run {i + 1}/{runs}")
        best_team, best_fitness, _, _ = solver.solve(pokemons, opponents=opponents)
        print(f"Best team: {best_team}")
        print(f"Best fitness: {best_fitness}")
        print(f"Best team stats sum: {best_team.get_stats_sum()}")

        best_teams.append(best_team)
        best_fitnesses.append(best_fitness)

    fitness_array = np.array(best_fitnesses, dtype=float)
    mean_fitness = float(np.mean(fitness_array))
    median_fitness = float(np.median(fitness_array))
    std_fitness = float(np.std(fitness_array))
    best_fitness_index = int(np.argmax(fitness_array))
    best_team = best_teams[best_fitness_index]
    best_fitness = best_fitnesses[best_fitness_index]

    print("Summary of multiple runs:")
    print(f"Mean fitness: {mean_fitness:.6f}")
    print(f"Median fitness: {median_fitness:.6f}")
    print(f"Std fitness: {std_fitness:.6f}")
    print(f"Best fitness: {best_fitness:.6f}")
    print(f"Best team: {best_team}")
    print(f"Best team stats sum: {best_team.get_stats_sum()}")

In [16]:
TEAM_SIZE = 6
OPP_LIMIT = 30
BUDGET = 50


pokemons = get_pokemons()
solver = SimulatedAnnealingPokemonSolver(
    initial_temperature=0.75,
    min_temperature=1e-3,
    alpha=0.995,
    iters_per_temp=30,
    max_evaluations=BUDGET,
    neighbor_replacements=1,
    opponents_limit=OPP_LIMIT,
    unique_types=True,
    seed=123,
    patience=10,
    restarts=1,
)

run_multiple_runs(pokemons, solver)

Running 8 runs of the SA solver...
Run 1/8
Best team: PokemonTeam(size=6, names=['Snorlax', 'Accelgor', 'Sylveon', 'Zoroark', 'Hariyama', 'Jynx'])
Best fitness: 0.24111479028697572
Best team stats sum: 1910
Run 2/8
Best team: PokemonTeam(size=6, names=['Sceptile', 'Scizor', 'Slowking', 'Oricorio', 'Krookodile', 'Slurpuff'])
Best fitness: 0.09869952087611226
Best team stats sum: 2201
Run 3/8
Best team: PokemonTeam(size=6, names=['Tyranitar', 'Ambipom', 'Beedrill', 'Phione', 'Garchomp', 'Granbull'])
Best fitness: 0.23835263835263834
Best team stats sum: 2241
Run 4/8
Best team: PokemonTeam(size=6, names=['Dragonite', 'Wailord', 'Slaking', 'Torterra', 'Machamp', 'Weavile'])
Best fitness: 0.2807307307307308
Best team stats sum: 2168
Run 5/8
Best team: PokemonTeam(size=6, names=['Electrode', 'Mandibuzz', 'Turtonator', 'Golurk', 'Slowbro', 'Glaceon'])
Best fitness: 0.09345859429366736
Best team stats sum: 2188
Run 6/8
Best team: PokemonTeam(size=6, names=['Donphan', 'Dunsparce', 'Tyranitar', 

In [19]:
def compare_to_random_search(
    pokemons: DataFrame[PokemonSchema],
    solver: SimulatedAnnealingPokemonSolver,
    baseline_solver: RandomSearchPokemonSolver,
    opponents: list[PokemonTeam] = None,
    runs: int = 8,
) -> None:
    print("Comparing SA solver to Random Search solver...")
    sa_model_results: list[tuple[PokemonTeam, float]] = []
    rs_model_results: list[tuple[PokemonTeam, float]] = []

    for i in range(runs):
        print(f"Run {i + 1}/{runs}")
        sa_best_team, sa_best_fitness, _, _ = solver.solve(
            pokemons, opponents=opponents
        )
        print(f"SA Best fitness: {sa_best_fitness}")
        rs_best_team, rs_best_fitness, _, _ = baseline_solver.solve(
            pokemons, opponents=opponents
        )
        print(f"RS Best fitness: {rs_best_fitness}")

        sa_model_results.append((sa_best_team, sa_best_fitness))
        rs_model_results.append((rs_best_team, rs_best_fitness))

    sa_fitnesses = np.array([fit for _, fit in sa_model_results], dtype=float)
    rs_fitnesses = np.array([fit for _, fit in rs_model_results], dtype=float)
    sa_mean_fitness = float(np.mean(sa_fitnesses))
    rs_mean_fitness = float(np.mean(rs_fitnesses))
    sa_std_fitness = float(np.std(sa_fitnesses))
    rs_std_fitness = float(np.std(rs_fitnesses))
    sa_wins = np.sum(sa_fitnesses > rs_fitnesses)
    rs_wins = np.sum(rs_fitnesses > sa_fitnesses)
    draws = runs - sa_wins - rs_wins
    print("Comparison summary:")
    print(f"SA Mean fitness: {sa_mean_fitness:.6f} ± {sa_std_fitness:.6f}")
    print(f"RS Mean fitness: {rs_mean_fitness:.6f} ± {rs_std_fitness:.6f}")
    print(f"SA wins: {sa_wins}, RS wins: {rs_wins}, Draws: {draws}")

In [20]:
pokemons = get_pokemons()
opponents = PokemonTeam.generate_unique_teams(
    pokemons,
    opponents_limit=OPP_LIMIT,
    max_attempts=OPP_LIMIT * 3,
    team_size=TEAM_SIZE,
    unique_types=True,
)

TRIALS = 50

random_search_solver = RandomSearchPokemonSolver(
    trials=TRIALS, opponents_limit=OPP_LIMIT, unique_types=True
)
solver = SimulatedAnnealingPokemonSolver(
    initial_temperature=0.75,
    min_temperature=1e-3,
    alpha=0.995,
    iters_per_temp=30,
    max_evaluations=BUDGET,
    neighbor_replacements=1,
    opponents_limit=OPP_LIMIT,
    unique_types=True,
    seed=123,
    patience=10,
    restarts=1,
)

compare_to_random_search(pokemons, solver, random_search_solver, opponents, runs=8)

Comparing SA solver to Random Search solver...
Run 1/8
SA Best fitness: 0.15370370370370373
RS Best fitness: 0.2564830272676684
Run 2/8
SA Best fitness: 0.14592901878914408
RS Best fitness: 0.25821669258683255
Run 3/8
SA Best fitness: 0.42974828375286034
RS Best fitness: 0.2081287506819422
Run 4/8
SA Best fitness: 0.12758169934640523
RS Best fitness: 0.2410394265232975
Run 5/8
SA Best fitness: 0.1473653049141504
RS Best fitness: 0.30040100250626567
Run 6/8
SA Best fitness: 0.3565768621236133
RS Best fitness: 0.3288571428571429
Run 7/8
SA Best fitness: 0.17178189994378865
RS Best fitness: 0.23866666666666664
Run 8/8
SA Best fitness: 0.15571236559139787
RS Best fitness: 0.2935374149659864
Comparison summary:
SA Mean fitness: 0.211050 ± 0.107333
RS Mean fitness: 0.265666 ± 0.036678
SA wins: 2, RS wins: 6, Draws: 0


In [24]:
def compare_to_hill_climb(
    pokemons: DataFrame[PokemonSchema],
    solver: SimulatedAnnealingPokemonSolver,
    baseline_solver: HillClimbingPokemonSolver,
    opponents: list[PokemonTeam] = None,
    runs: int = 8,
) -> None:
    print("Comparing SA solver to Hill Climbing solver...")

    sa_model_results: list[tuple[PokemonTeam, float]] = []
    hc_model_results: list[tuple[PokemonTeam, float]] = []

    for i in range(runs):
        starting_team = PokemonTeam.generate_team(
            pokemons, team_size=TEAM_SIZE, unique_types=True
        )
        print(f"Run {i + 1}/{runs}")
        sa_best_team, sa_best_fitness, _, _ = solver.solve(
            pokemons, opponents=opponents, start_team=starting_team
        )
        print(f"SA Best fitness: {sa_best_fitness}")
        hc_best_team, hc_best_fitness, _, _ = baseline_solver.solve(
            pokemons, opponents=opponents, start_team=starting_team
        )
        print(f"HC Best fitness: {hc_best_fitness}")

        sa_model_results.append((sa_best_team, sa_best_fitness))
        hc_model_results.append((hc_best_team, hc_best_fitness))

    sa_fitnesses = np.array([fit for _, fit in sa_model_results], dtype=float)
    hc_fitnesses = np.array([fit for _, fit in hc_model_results], dtype=float)
    sa_mean_fitness = float(np.mean(sa_fitnesses))
    hc_mean_fitness = float(np.mean(hc_fitnesses))
    sa_std_fitness = float(np.std(sa_fitnesses))
    hc_std_fitness = float(np.std(hc_fitnesses))
    sa_wins = np.sum(sa_fitnesses > hc_fitnesses)
    hc_wins = np.sum(hc_fitnesses > sa_fitnesses)
    draws = runs - sa_wins - hc_wins

    print("Comparison summary:")
    print(f"SA Mean fitness: {sa_mean_fitness:.6f} ± {sa_std_fitness:.6f}")
    print(f"HC Mean fitness: {hc_mean_fitness:.6f} ± {hc_std_fitness:.6f}")
    print(f"SA wins: {sa_wins}, HC wins: {hc_wins}, Draws: {draws}")

    best_sa_team, best_sa_fitness = sa_model_results[int(np.argmax(sa_fitnesses))]
    best_hc_team, best_hc_fitness = hc_model_results[int(np.argmax(hc_fitnesses))]

    print("Best SA team:")
    print(
        f"Team: {best_sa_team}",
        f"Fitness: {best_sa_fitness}, Stats sum: {best_sa_team.get_stats_sum()}",
    )
    print("Best HC team:")
    print(
        f"Team: {best_hc_team}",
        f"Fitness: {best_hc_fitness}, Stats sum: {best_hc_team.get_stats_sum()}",
    )

In [22]:
pokemons = get_pokemons()
opponents = PokemonTeam.generate_unique_teams(
    pokemons,
    opponents_limit=OPP_LIMIT,
    max_attempts=OPP_LIMIT * 3,
    team_size=TEAM_SIZE,
    unique_types=True,
)

TRIALS = 50

random_search_solver = RandomSearchPokemonSolver(
    trials=TRIALS, opponents_limit=OPP_LIMIT, unique_types=True
)
solver = SimulatedAnnealingPokemonSolver(
    initial_temperature=0.75,
    min_temperature=1e-3,
    alpha=0.995,
    iters_per_temp=30,
    max_evaluations=BUDGET,
    neighbor_replacements=1,
    opponents_limit=OPP_LIMIT,
    unique_types=True,
    seed=123,
    patience=10,
    restarts=0,
)

hill_climbing_solver = HillClimbingPokemonSolver(
    max_evaluations=BUDGET,
    neighbors_per_step=30,
    neighbor_replacements=1,
    opponents_limit=OPP_LIMIT,
    unique_types=True,
    seed=123,
    restarts=0,
)

compare_to_hill_climb(pokemons, solver, hill_climbing_solver, opponents, runs=8)

Comparing SA solver to Hill Climbing solver...
Run 1/8
SA Best fitness: 0.14969021065675342
HC Best fitness: 0.31238845144356947
Run 2/8
SA Best fitness: 0.22037037037037038
HC Best fitness: 0.286297957149975
Run 3/8
SA Best fitness: 0.1315985130111524
HC Best fitness: 0.16946386946386946
Run 4/8
SA Best fitness: 0.09915857605177993
HC Best fitness: 0.2171128608923885
Run 5/8
SA Best fitness: 0.1310391363022942
HC Best fitness: 0.219626168224299
Run 6/8
SA Best fitness: 0.12691846522781774
HC Best fitness: 0.23182912154031288
Run 7/8
SA Best fitness: 0.21399399399399396
HC Best fitness: 0.14794352363413138
Run 8/8
SA Best fitness: 0.18778816199376946
HC Best fitness: 0.16592455163883735
Comparison summary:
SA Mean fitness: 0.157570 ± 0.041582
HC Mean fitness: 0.218823 ± 0.054473
SA wins: 2, HC wins: 6, Draws: 0


In [25]:
pokemons = get_pokemons()

TEAM_SIZE = 6
OPP_LIMIT = 50
BUDGET = 150


opponents = PokemonTeam.generate_unique_teams(
    pokemons,
    opponents_limit=OPP_LIMIT,
    max_attempts=OPP_LIMIT * 3,
    team_size=TEAM_SIZE,
    unique_types=True,
)


random_search_solver = RandomSearchPokemonSolver(
    trials=TRIALS, opponents_limit=OPP_LIMIT, unique_types=True
)
solver = SimulatedAnnealingPokemonSolver(
    initial_temperature=0.75,
    min_temperature=1e-3,
    alpha=0.995,
    iters_per_temp=30,
    max_evaluations=BUDGET,
    neighbor_replacements=1,
    opponents_limit=OPP_LIMIT,
    unique_types=True,
    seed=123,
    patience=10,
    restarts=0,
)

hill_climbing_solver = HillClimbingPokemonSolver(
    max_evaluations=BUDGET,
    neighbors_per_step=30,
    neighbor_replacements=1,
    opponents_limit=OPP_LIMIT,
    unique_types=True,
    seed=123,
    restarts=0,
)

compare_to_hill_climb(pokemons, solver, hill_climbing_solver, opponents, runs=8)

Comparing SA solver to Hill Climbing solver...
Run 1/8
SA Best fitness: 0.08645833333333335
HC Best fitness: 0.33071641791044776
Run 2/8
SA Best fitness: 0.2519677419354839
HC Best fitness: 0.30737931034482757
Run 3/8
SA Best fitness: 0.19676521739130437
HC Best fitness: 0.379127988748242
Run 4/8
SA Best fitness: 0.13607003891050581
HC Best fitness: 0.3304918032786885
Run 5/8
SA Best fitness: 0.2334622144112478
HC Best fitness: 0.4657622377622378
Run 6/8
SA Best fitness: 0.08804166666666667
HC Best fitness: 0.37570186335403727
Run 7/8
SA Best fitness: 0.13175757575757577
HC Best fitness: 0.31877216916780354
Run 8/8
SA Best fitness: 0.16844537815126048
HC Best fitness: 0.3166816143497757
Comparison summary:
SA Mean fitness: 0.161621 ± 0.058324
HC Mean fitness: 0.353079 ± 0.049356
SA wins: 0, HC wins: 8, Draws: 0
Best SA team:
Team: PokemonTeam(size=6, names=['Dewgong', 'Heatmor', 'Gogoat', 'Mothim', 'Garchomp', 'Hariyama']) Fitness: 0.2519677419354839, Stats sum: 2057
Best HC team:
Team

In [26]:
pokemons = get_pokemons()

TEAM_SIZE = 6
OPP_LIMIT = 200
BUDGET = 250


opponents = PokemonTeam.generate_unique_teams(
    pokemons,
    opponents_limit=OPP_LIMIT,
    max_attempts=OPP_LIMIT * 3,
    team_size=TEAM_SIZE,
    unique_types=True,
)


random_search_solver = RandomSearchPokemonSolver(
    trials=TRIALS, opponents_limit=OPP_LIMIT, unique_types=True
)
solver = SimulatedAnnealingPokemonSolver(
    initial_temperature=1,
    min_temperature=1e-34,
    alpha=0.995,
    iters_per_temp=50,
    max_evaluations=BUDGET,
    neighbor_replacements=1,
    opponents_limit=OPP_LIMIT,
    unique_types=True,
    seed=123,
    patience=300,
    restarts=2,
)

hill_climbing_solver = HillClimbingPokemonSolver(
    max_evaluations=BUDGET,
    neighbors_per_step=30,
    neighbor_replacements=1,
    opponents_limit=OPP_LIMIT,
    unique_types=True,
    seed=123,
    patience=300,
    restarts=2,
)

compare_to_hill_climb(pokemons, solver, hill_climbing_solver, opponents, runs=8)

Comparing SA solver to Hill Climbing solver...
Run 1/8
SA Best fitness: 0.25804777594728173
HC Best fitness: 0.410251497005988
Run 2/8
SA Best fitness: 0.2856919642857143
HC Best fitness: 0.4979959239130434
Run 3/8
SA Best fitness: 0.35015006821282396
HC Best fitness: 0.49086561743341406
Run 4/8
SA Best fitness: 0.1877534791252485
HC Best fitness: 0.436855421686747
Run 5/8
SA Best fitness: 0.1902054794520548
HC Best fitness: 0.3701649175412294
Run 6/8
SA Best fitness: 0.28984349258649095
HC Best fitness: 0.39900271370420626
Run 7/8
SA Best fitness: 0.2313403880070547
HC Best fitness: 0.442192118226601
Run 8/8
SA Best fitness: 0.2405046583850932
HC Best fitness: 0.45615789473684215
Comparison summary:
SA Mean fitness: 0.254192 ± 0.050872
HC Mean fitness: 0.437936 ± 0.041284
SA wins: 0, HC wins: 8, Draws: 0
Best SA team:
Team: PokemonTeam(size=6, names=['Luxray', 'Mudsdale', 'Amoonguss', 'Meowstic', 'Mandibuzz', 'Blissey']) Fitness: 0.35015006821282396, Stats sum: 1896
Best HC team:
Team

In [ ]:
pokemons = get_pokemons()

TEAM_SIZE = 6
OPP_LIMIT = 80
BUDGET = 400


opponents = PokemonTeam.generate_unique_teams(
    pokemons,
    opponents_limit=OPP_LIMIT,
    max_attempts=OPP_LIMIT * 3,
    team_size=TEAM_SIZE,
    unique_types=True,
)


solver = SimulatedAnnealingPokemonSolver(
    initial_temperature=0.45,
    min_temperature=1e-3,
    alpha=0.995,
    iters_per_temp=20,
    max_evaluations=BUDGET,
    neighbor_replacements=1,
    opponents_limit=OPP_LIMIT,
    unique_types=True,
    seed=123,
    patience=300,
    restarts=0,
)

run_multiple_runs(pokemons, solver, runs=8, opponents=opponents)

Running 8 runs of the SA solver...
Run 1/8
Best team: PokemonTeam(size=6, names=['Sableye', 'Snorlax', 'Gogoat', 'Whiscash', 'Salamence', 'Delphox'])
Best fitness: 0.2644779771615008
Best team stats sum: 2238
Run 2/8
Best team: PokemonTeam(size=6, names=['Kommo-o', 'Golisopod', 'Mudsdale', 'Avalugg', 'Tangrowth', 'Blissey'])
Best fitness: 0.3524107142857143
Best team stats sum: 2226
Run 3/8
Best team: PokemonTeam(size=6, names=['Phione', 'Tangrowth', 'Blissey', 'Krookodile', 'Primeape', 'Ninetales'])
Best fitness: 0.3157747005988024
Best team stats sum: 1885
Run 4/8
Best team: PokemonTeam(size=6, names=['Crabominable', 'Blissey', 'Umbreon', 'Carracosta', 'Weezing', 'Metagross'])
Best fitness: 0.3058183183183183
Best team stats sum: 2197
Run 5/8
Best team: PokemonTeam(size=6, names=['Blaziken', 'Rampardos', 'Amoonguss', 'Clawitzer', 'Magnezone', 'Blissey'])
Best fitness: 0.3623180494905386
Best team stats sum: 2115
Run 6/8
Best team: PokemonTeam(size=6, names=['Aurorus', 'Gogoat', 'Hipp

In [7]:
pokemons = get_pokemons()

TEAM_SIZE = 6
OPP_LIMIT = 200
BUDGET = 600


opponents = PokemonTeam.generate_unique_teams(
    pokemons,
    opponents_limit=OPP_LIMIT,
    max_attempts=OPP_LIMIT * 3,
    team_size=TEAM_SIZE,
    unique_types=True,
)


solver = SimulatedAnnealingPokemonSolver(
    initial_temperature=0.45,
    min_temperature=1e-3,
    alpha=0.6,
    iters_per_temp=30,
    max_evaluations=BUDGET,
    neighbor_replacements=1,
    opponents_limit=OPP_LIMIT,
    unique_types=True,
    seed=123,
    patience=300,
    restarts=2,
)

run_multiple_runs(pokemons, solver, runs=8, opponents=opponents)

Running 8 runs of the SA solver...
Run 1/8
Best team: PokemonTeam(size=6, names=['Tyranitar', 'Blaziken', 'Mudsdale', 'Vaporeon', 'Amoonguss', 'Musharna'])
Best fitness: 0.380796875
Best team stats sum: 2336
Run 2/8
Best team: PokemonTeam(size=6, names=['Gogoat', 'Rhyperior', 'Ampharos', 'Blissey', 'Drifblim', 'Wailord'])
Best fitness: 0.5229180509413068
Best team stats sum: 1963
Run 3/8
Best team: PokemonTeam(size=6, names=['Tyranitar', 'Emboar', 'Hippowdon', 'Musharna', 'Slaking', 'Lapras'])
Best fitness: 0.4252170868347339
Best team stats sum: 2359
Run 4/8
Best team: PokemonTeam(size=6, names=['Hippowdon', 'Abomasnow', 'Ampharos', 'Swalot', 'Alomomola', 'Wobbuffet'])
Best fitness: 0.4363257065948856
Best team stats sum: 2053
Run 5/8
Best team: PokemonTeam(size=6, names=['Tyranitar', 'Amoonguss', 'Hippowdon', 'Snorlax', 'Alomomola', 'Hariyama'])
Best fitness: 0.48051201011378
Best team stats sum: 2089
Run 6/8
Best team: PokemonTeam(size=6, names=['Blaziken', 'Tyranitar', 'Goodra', 'M